# 深度学习中的激活函数
## 激活函数的定义与作用
Bengio教授在 ICML 2016 的文章[1]中给出了激活函数的定义：激活函数是映射 h:R→R，且几乎处处可导。

神经网络中激活函数的主要作用是提供网络的非线性建模能力，如不特别说明，激活函数一般而言是非线性函数。假设一个示例神经网络中仅包含线性卷积和全连接运算，那么该网络仅能够表达线性映射，即便增加网络的深度也依旧还是线性映射，难以有效建模实际环境中非线性分布的数据。加入（非线性）激活函数之后，深度神经网络才具备了分层的非线性映射学习能力。

## 激活函数的历史发展与近期进展
### Sigmoid
Sigmoid 是使用范围最广的一类激活函数，具有指数函数形状 。正式定义为：

$f(x)=\frac{1}{1+e^{-x}}$

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwgicU8u7Ela9RhgOGQZeibic1dOHJouFfz0GEYJfo3EEn0LHuKfdJ9XichRA/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

可见，sigmoid 在定义域内处处可导，且两侧导数逐渐趋近于0，即：

$lim_{x \to \infty}f'(x)=0$

Bengio 教授等[1]将具有这类性质的激活函数定义为`软饱和激活函数`。与极限的定义类似，饱和也分为左饱和:

$lim_{x \to -\infty}f'(x)=0$

右饱和：

$lim_{x \to +\infty}f'(x)=0$

与软饱和相对的是硬饱和激活函数，即：

f'(x)=0, 当 |x| > c,其中c为常数。

同理，硬饱和也分为左饱和和右饱和。常见的 ReLU 就是一类左侧硬饱和激活函数。

Sigmoid 的软饱和性，使得深度神经网络在二三十年里一直难以有效的训练，是阻碍神经网络发展的重要原因。具体来说，由于在后向传递过程中，sigmoid向下传导的梯度包含了一个f'(x) 因子（sigmoid关于输入的导数），因此一旦输入落入饱和区，f'(x) 就会变得接近于0，导致了向底层传递的梯度也变得非常小。此时，网络参数很难得到有效训练。这种现象被称为`梯度消失`。一般来说， sigmoid 网络在 5 层之内就会产生梯度消失现象[2]。梯度消失问题至今仍然存在，但被新的优化方法有效缓解了，例如DBN中的分层预训练，Batch Normalization的逐层归一化，Xavier和MSRA权重初始化等代表性技术。

Sigmoid 的饱和性虽然会导致梯度消失，但也有其有利的一面。例如它在物理意义上最为接近生物神经元。 (0, 1) 的输出还可以被表示作概率，或用于输入的归一化，代表性的如Sigmoid交叉熵损失函数。

### tanh
$tanh(x)=\frac{1-e^{-2x}}{1+e^{-2x}}$

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwg4BqZrl2DQeX0GzDVCiam3YNEcHJ0GtmY954PvIVwzibuGibFgMBmJelicA/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

可见，tanh(x)=2sigmoid(2x)-1，也具有软饱和性。Xavier在文献[2]中分析了sigmoid与tanh的饱和现象及特点，具体见原论文。此外，文献 [3] 中提到tanh 网络的收敛速度要比sigmoid快。因为 tanh 的输出均值比 sigmoid 更接近 0，SGD会更接近 natural gradient[4]（一种二次优化技术），从而降低所需的迭代次数。

### ReLU
虽然2006年Hinton教授提出通过分层无监督预训练解决深层网络训练困难的问题，但是深度网络的直接监督式训练的最终突破，最主要的原因是采用了新型激活函数ReLU[5, 6]。与传统的sigmoid激活函数相比，ReLU能够有效缓解梯度消失问题，从而直接以监督的方式训练深度神经网络，无需依赖无监督的逐层预训练，这也是2012年深度卷积神经网络在ILSVRC竞赛中取得里程碑式突破的重要原因之一。

ReLU的 正式定义为：

$$
y=\left\{\begin{matrix}
x & if\ x \ge 0 \\ 
0 & if\ x < 0
\end{matrix}\right.
$$

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwgOo9xqTatEbyZCicLX8UzWQT2Zn36UFnicmtvKKAickJBguuwznNPJKxSQ/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

可见，ReLU 在x<0 时硬饱和。由于 x>0时导数为 1，所以，ReLU 能够在x>0时保持梯度不衰减，从而缓解梯度消失问题。但随着训练的推进，部分输入会落入硬饱和区，导致对应权重无法更新。这种现象被称为`“神经元死亡”`。

ReLU还经常被“诟病”的一个问题是输出具有偏移现象[7]，即输出均值恒大于零。偏移现象和 神经元死亡会共同影响网络的收敛性。本文作者公开在arxiv的文章[8]中的实验表明，如果不采用Batch Normalization，即使用 MSRA 初始化30层以上的ReLU网络，最终也难以收敛。相对的，PReLU和ELU网络都能顺利收敛，这两种改进的激活函数将在后面介绍。实验所用代码见 https://github.com/Coldmooon/Code-for-MPELU/ 。

ReLU另外一个性质是提供神经网络的稀疏表达能力，在Bengio教授的Deep Sparse Rectifier Neural Network[6]一文中被认为是ReLU带来网络性能提升的原因之一。但后来的研究发现稀疏性并非性能提升的必要条件，文献 RReLU [9]也指明了这一点。

PReLU[10]、ELU[7]等激活函数不具备这种稀疏性，但都能够提升网络性能。本文作者在文章[8]中给出了一些实验比较结果。首先，在cifar10上采用NIN网络，实验结果为 PReLU > ELU > ReLU，稀疏性并没有带来性能提升。其次，在 ImageNet上采用类似于[11] 中model E的15 层网络，实验结果则是ReLU最好。为了验证是否是稀疏性的影响，以 LReLU [12]为例进一步做了四次实验，负半轴的斜率分别为1，0.5，0.25,  0.1，需要特别说明的是，当负半轴斜率为1时，LReLU退化为线性函数，因此性能损失最大。实验结果展现了斜率大小与网络性能的一致性。综合上述实验可知，ReLU的稀疏性与网络性能之间并不存在绝对正负比关系。

LReLU斜率a | ImageNet top-1分类精度(%)
---------|----------------------
0 | 62.34
0.1 | 62.08
0.25 | 61.46
0.5 | 57.24
1 | 39.73

### PReLU
PReLU [10]是ReLU 和 LReLU的改进版本，具有非饱和性：

$$
f(y_i)=\left\{\begin{matrix}
y_i & if\ y_i > 0 \\ 
a_iy_i & if\ y_i \le 0
\end{matrix}\right.
$$

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwgJlRHlBMdeAkezy5X6TXxxqecZ7IOSrBXBh1AsEmRO7ZT31kJTkoTOA/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

与LReLU相比，PReLU中的负半轴斜率a可学习而非固定。原文献建议初始化a为0.25，不采用正则。个人认为，是否采用正则应当视具体的数据库和网络，通常情况下使用正则能够带来性能提升。

虽然PReLU 引入了额外的参数，但基本不需要担心过拟合。例如，在上述cifar10+NIN实验中， PReLU比ReLU和ELU多引入了参数，但也展现了更优秀的性能。所以实验中若发现网络性能不好，建议从其他角度寻找原因。

与ReLU相比，PReLU收敛速度更快。因为PReLU的输出更接近0均值，使得SGD更接近natural gradient。证明过程参见原文[10]。

此外，作者在ResNet 中采用ReLU，而没有采用新的PReLU。这里给出个人浅见，不一定正确，仅供参考。首先，在上述LReLU实验中，负半轴斜率对性能的影响表现出一致性。对PReLU采用正则将激活值推向0也能够带来性能提升。这或许表明，小尺度或稀疏激活值对深度网络的影响更大。其次，ResNet中包含单位变换和残差两个分支。残差分支用于学习对单位变换的扰动。如果单位变换是最优解，那么残差分支的扰动应该越小越好。这种假设下，小尺度或稀疏激活值对深度网络的影响更大。此时，ReLU或许是比PReLU更好的选择。

### RReLU
数学形式与PReLU类似，但RReLU[9]是一种非确定性激活函数，其参数是随机的。这种随机性类似于一种噪声，能够在一定程度上起到正则效果。作者在cifar10/100上观察到了性能提升。

### Maxout
Maxout[13]是ReLU的推广，其发生饱和是一个零测集事件（measure zero event）。正式定义为：

$max(w_1^Tx+b_1,w_2^Tx+b_2, \cdots,w_n^Tx+b_n)$

Maxout网络能够近似任意连续函数，且当$w_2,b_2,…,w_n,b_n$为0时，退化为ReLU。 其实，Maxout的思想在视觉领域存在已久。例如，在HOG特征里有这么一个过程：计算三个通道的梯度强度，然后在每一个像素位置上，仅取三个通道中梯度强度最大的数值，最终形成一个通道。这其实就是Maxout的一种特例。

Maxout能够缓解梯度消失，同时又规避了ReLU神经元死亡的缺点，但增加了参数和计算量。

### ELU
ELU[7]融合了sigmoid和ReLU，具有左侧软饱性。其正式定义为：

$$
f(x)=\left\{\begin{matrix}
x & if\ x > 0 \\ 
\alpha(exp(x)-1) & if\ x \le 0
\end{matrix}\right.
$$

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwgESEL4k1PBRIRTHxnloT6GHbicicrjcafVvZIgb5IInG8UBR0etjNibTUA/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

右侧线性部分使得ELU能够缓解梯度消失，而左侧软饱能够让ELU对输入变化或噪声更鲁棒。ELU的输出均值接近于零，所以收敛速度更快。经本文作者实验，ELU的收敛性质的确优于ReLU和PReLU。在cifar10上，ELU 网络的loss 降低速度更快；在 ImageNet上，不加 Batch Normalization 30 层以上的 ReLU 网络会无法收敛，PReLU网络在MSRA的Fan-in （caffe ）初始化下会发散，而 ELU 网络在Fan-in/Fan-out下都能收敛 。实验代码见 https://github.com/Coldmooon/Code-for-MPELU/。

论文的另一个重要贡献是分析了Bias shift 现象与激活值的关系，证明了降低Bias shift 等价于把激活值的均值推向0。

### Noisy Activation Functions
Bengio教授在ICML 2016 提出了一种激活策略[1]，可用于多种软饱和激活函数，例如 sigmoid和 tanh。

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwgMbp9SUBuy3YQibDhicUuyp2BbGN89AmK6u2GAQHLSbibGuVmCYc7Aw9yw/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwgOat8D4Xl2eGvPDwPQPvl1axKLWDhYyTBrm21lPsvJ6QRuc7LdicRAIQ/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

当激活函数发生饱和时， 网络参数还能够在两种动力下继续更新：正则项梯度和噪声梯度。引入适当的噪声能够扩大SGD的参数搜索范围，从而有机会跳出饱和区。在激活函数中引入噪声的更早工作可追溯到[5]，但文献[5]的工作并不考虑噪声引入的时间和大小。本篇的特点在于，只在饱和区才引入噪声，且噪声量与饱和程度相关——原式与泰勒展开式一次项之差 δ。算法1中g表示sigmoid，用于归一化 δ。注意，ReLU的 δ恒为0，无法直接加噪声，所以作者把噪声加在了输入上。

### CReLU
CReLU [14]是Wenling Shang 发表在 ICML 2016的工作，本篇同样提出了一种激活策略:

$\forall x \in \mathbb{R}, \rho_c(x) \triangleq ([x]_{+},[-x]_{+})$

其中，[] 表示 ReLU（其他亦可）。

作者在观察第一层滤波器（filter）时发现，滤波器相位具有成对现象（pair-grouping phenomenon）。这一发现揭示了网络的底层学到了一些冗余滤波器来提取输入的正负相位信息的可能性。因此可以考虑采用适当的操作移除这些冗余滤波器。对此，作者提出了CReLU，将激活函数的输入额外做一次取反，等价于将输入相位旋转180°。这种策略可以看作在网络中加入相位的先验。实验在cifar10上观察到能以更少的参数获得性能提升。

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwgiashzicpV37JUOMoicydWDysS0Phlnf0OunCDEA9IRFOkYfKJepTfQbmw/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

使用CReLU时，要有意识的将滤波器数量减半，否则， 网络参数变为2倍。

### MPELU
MPELU[8]是我们组的工作，将分段线性与ELU统一到了一种形式下。在NIN+CIFAR10，本文作者发现ELU与LReLU性能一致，而与PReLU差距较大。经过分析，ELU泰勒展开的一次项就是LReLU。当在ELU前加入BN让输入集中在0均值附近， 则ELU与LReLU之差——泰勒展开高次项会变小，粗略估计，约55.57%的激活值误差小于0.01。因此，受PReLU启发，令α可学习能够提高性能。此外，引入参数β能够进一步控制ELU的函数形状。正式定义为：

$$
f(y_i)=\left\{\begin{matrix}
y_i & if\ y_i > 0 \\ 
\alpha_c(e^{\beta_cy_i}-1) & if\ x \le 0
\end{matrix}\right.
$$

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwgKkQOxwUjo8BvKzROLWkXuTAMrLydcpBt1M9oOShicWOA1M8fa3bMcBA/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

α 和 β可以使用正则。α, β 固定为1时，MPELU 退化为 ELU； β 固定为很小的值时，MPELU 近似为 PReLU；当α=0，MPELU 等价于 ReLU。

MPELU 的优势在于同时具备 ReLU、PReLU和 ELU的优点。首先，MPELU具备ELU的收敛性质，能够在无 Batch Normalization 的情况下让几十层网络收敛。其次，作为一般化形式， MPELU较三者的推广能力更强。简言之，MPELU = max(ReLU, PReLU, ELU)。

![](http://mmbiz.qpic.cn/mmbiz_jpg/xRp3sibCWzgEibibmQgFot91Viawy89yZmwg5NwH2JXibEib3ubmPnOP19oAghyusbtsXAibYOTSNPmvI6FAH4wUlJjnA/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)

当前对ELU网络普遍采用的初始化方法是 MSRA。这在实际中是可行的，只是不具备理论解释性。我们的工作利用泰勒公式和MSRA的推导过程，为ELU网络初始化提供了理论解释。此外，Dmytro 提出了 LSUV[15]，理论上可以用于 ELU/MPELU 的初始化。但在30/52层ELU网络上，发现 LSUV 会导致ELU网络在几次迭代之内发散，网络文件见https://github.com/Coldmooon/Code-for-MPELU/。

## Softmax
wiki百科对softmax函数的定义：

    softmax is a generalization of logistic function that “squashes”(maps) a K-dimensional vector z of arbitrary real values to a K-dimensional vector σ(z) of real values in the range (0, 1) that add up to 1.

这句话既表明了softmax函数与logistic函数的关系，也同时阐述了softmax函数的本质就是将一个K维的任意实数向量压缩（映射）成另一个K维的实数向量，其中向量中的每个元素取值都介于（0，1）之间。

softmax函数形式如下： 

$\sigma(z)_j=\frac{e^{z_j}}{\sum_{k=1}^K e^{z_k}}$


1. Gulcehre, C., et al., Noisy Activation Functions, in ICML 2016. 2016.
1. Glorot, X. and Y. Bengio. Understanding the difficulty of training deep feedforward neural networks. AISTATS 2010.
1. LeCun, Y., et al., Backpropagation applied to handwritten zip code recognition. Neural computation, 1989. 1(4): p. 541-551.
1. Amari, S.-I., Natural gradient works efficiently in learning. Neural computation, 1998. 10(2): p. 251-276.
1. Nair, V. and G.E. Hinton. Rectified linear units improve Restricted Boltzmann machines. ICML 2010.
1. Glorot, X., A. Bordes, and Y. Bengio. Deep Sparse Rectifier Neural Networks.AISTATS 2011.
1. Djork-Arné Clevert, T.U., Sepp Hochreiter. Fast and Accurate Deep Network Learning by Exponential Linear Units (ELUs). ICLR 2016.
1. Li, Y., et al., Improving Deep Neural Network with Multiple Parametric Exponential Linear Units. arXiv preprint arXiv:1606.00305, 2016.
1. Xu, B., et al. Empirical Evaluation of Rectified Activations in Convolutional Network. ICML Deep Learning Workshop 2015.
1. He, K., et al. Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification. ICCV 2015.
1. He, K. and J. Sun Convolutional Neural Networks at Constrained Time Cost. CVPR 2015.
1. Maas, A.L., Awni Y. Hannun, and Andrew Y. Ng. Rectifier nonlinearities improve neural network acoustic models. in ICML 2013.
1. Goodfellow, I.J., et al. Maxout Networks.  ICML 2013..
1. Shang, W., et al., Understanding and Improving Convolutional Neural Networks via Concatenated Rectified Linear Units.ICML 2016.
1. Mishkin, D. and J. Matas All you need is a good init. ICLR 2016.